In [27]:
import keras
from keras.layers import Dense, Dropout
import librosa
import os
import scipy
import numpy as np
import pandas as pd
import scipy

In [49]:
os.getcwd()

'C:\\Users\\Ravindran'

In [11]:
model = keras.models.load_model(r"C:\Users\Ravindran\Downloads\Emotion_Voice_Detection_Model.h5")
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_7 (Conv1D)            (None, 216, 128)          768       
_________________________________________________________________
activation_8 (Activation)    (None, 216, 128)          0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 216, 128)          82048     
_________________________________________________________________
activation_9 (Activation)    (None, 216, 128)          0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 216, 128)          0         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 27, 128)           0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 27, 128)           82048     
__________

In [12]:
model.layers.pop()
model.layers.pop() #Remove last two layers
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_7 (Conv1D)            (None, 216, 128)          768       
_________________________________________________________________
activation_8 (Activation)    (None, 216, 128)          0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 216, 128)          82048     
_________________________________________________________________
activation_9 (Activation)    (None, 216, 128)          0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 216, 128)          0         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 27, 128)           0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 27, 128)           82048     
__________

In [15]:
for layer in model.layers:
    layer.trainable =False
model.compile(loss='mse', optimizer='adam')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_7 (Conv1D)            (None, 216, 128)          768       
_________________________________________________________________
activation_8 (Activation)    (None, 216, 128)          0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 216, 128)          82048     
_________________________________________________________________
activation_9 (Activation)    (None, 216, 128)          0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 216, 128)          0         
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 27, 128)           0         
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 27, 128)           82048     
__________

In [44]:
def getMFCCVectors(dir_path):
    col_names = ["File"]
    for i in range(216):
        col_names.append("f_{0}".format(i))
    df = pd.DataFrame(columns=col_names)
    files = mylist= os.listdir(dir_path)
    for wav_file in files:
            X, sample_rate = librosa.load(dir_path + '\\' + wav_file, res_type='kaiser_fast',duration=2.5,sr=22050*2,offset=0.5)
            sample_rate = np.array(sample_rate)
            mfccs = np.mean(librosa.feature.mfcc(y=X,sr=sample_rate,n_mfcc=13),axis=0)
            feature = list(mfccs)
            feature.insert(0,wav_file)
            #feature = np.insert(feature,0,wav_file,axis=0)
            df = df.append(pd.Series(feature,index=col_names),ignore_index=True)
    return df
df = getMFCCVectors(r"C:\Users\Ravindran\Downloads\wavfiles")

In [45]:
df.head()

,File,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,...,f_206,f_207,f_208,f_209,f_210,f_211,f_212,f_213,f_214,f_215
0,ENG_F.wav,-9.634951,-9.795591,-7.912519,-7.362560,-5.170220,-3.271047,-5.071285,-5.283034,-6.875960,...,-5.866347,-4.931830,-3.612926,-2.747670,-2.630764,-2.393044,-3.350479,-4.923983,-5.570993,-4.840384
1,ENG_M.wav,-17.748975,-12.823216,-10.532903,-4.897205,-1.273660,-1.535959,-3.151714,-3.126618,-2.579296,...,-1.669479,-0.630059,1.019179,2.483909,2.777182,3.832421,3.937623,3.368763,4.578668,7.699327
2,female.wav,-3.672385,-7.998870,-31.124778,-27.719597,-28.442601,-28.604341,-24.796864,-23.330125,-24.092907,...,-21.862271,-21.946479,-24.770703,-25.932881,-27.643926,-29.979669,-32.414012,-33.992713,-35.369626,-36.156626
3,male.wav,-33.068170,-31.963136,-31.209574,-31.493340,-32.645038,-32.751828,-31.306887,-22.045961,-15.669056,...,-18.641243,-15.943071,-15.364504,-17.837066,-19.124393,-18.658302,-19.753506,-18.602940,-19.642421,-25.735081
